In [ ]:
import cv2
import numpy as np
import tensorflow as tf

### 25200개의 바운딩박스에서 object conf.threshold 값을 기준으로 좋은 바운딩박스만 걸러내기

1. 앞 단계와 동일하게 모델 로드 후 테스트 이미지를 넣고 추론을 실시합니다.

In [ ]:
model = tf.saved_model.load("backup/best_saved_model")
src = cv2.imread("test_images/test_image.png")
src_converted = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
src_resized = cv2.resize(src_converted, (640, 640))
src_resized = src_resized.reshape((1, 640, 640, 3))
src_resized = src_resized/255.
pred = model(src_resized)
pred = pred[0][0].numpy() # -> 추론결과를 담은 (25200, 7) 모양의 numpy ndarray

2. 작업 용이성을 위해 cxcywh를 xyxy로 변환하는 함수를 미리 만들어 놓습니다.

In [ ]:
def getXyxy(cxcywh:list): #[cx, cy, w, h]로 된 리스트를 입력으로 받음
    x1 = cxcywh[0] - (cxcywh[2]/2) # -> cx - (w/2)
    y1 = cxcywh[1] - (cxcywh[3]/2) # -> cy - (h/2)
    x2 = x1 + cxcywh[2] # -> x1 + w
    y2 = y1 + cxcywh[3] # -> y1 + h
    return [x1, y1, x2, y2]

3. 추론 결과를 입력으로 받은 후 obj conf.로 cutoff한 후(threshold), 클래스별로 결과를 분리해 출력해 주는 함수를 만들겠습니다.

In [ ]:
def getCandidatesByClass(pred, conf_thres=0.25): #pred : (25200, 7)
    #pred가 담고 있는 클래스의 개수가 몇개인지 계산해 class_len에 담아 놓는다.
    #pred[0] -> 25200개의 추론 결과 중 첫 번째 행
    #pred[0].shape -> (7,)
    #pred[0].shape[0] -> 7
    class_len = pred[0].shape[0] - 5 #5개 열(cx, cy, w, h, obj conf.)를 뺀 나머지 열이 클래스의 개수
    candidate_list = [] #바운딩박스 후보군을 담아놓을 빈 리스트를 하나 생성

    #클래스별로 바운딩박스 결과를 분리해 담기 위해, candidate_list에 클래스별 딕셔너리를 추가
    for class_no in range(class_len):
        candidate_list.append({
            "class_no" : class_no, #클래스 번호 (0, 1)
            "bboxes(xyxy)" : [] #여기에 들어갈 값은 아래에서 추가할 것이므로 빈 리스트로 넣어 놓음
        })
    
    #25200 각 행으로 접근하여 conf_thres 값보다 높은 obj conf. 값을 가진 bbox 결과만 추려냄
    for item in pred:
        #클래스별 확률값을 담고 있는 배열만 슬라이싱(item[5:])한 후 최대값이 위치한 곳의 index 번호 가져오기(argmax)
        this_class_no = item[5:].argmax() # -> 최대값이 위치한 곳의 index = class_no
        this_conf = item[4] # obj conf.
        # this_conf = item[4] * item[5 + this_class_no] # 좀 더 엄밀한 검출을 하고 싶다면 obj conf와 cls conf를 곱해 conf값을 계산하면 됨
        if this_conf >= conf_thres: #this_conf가 conf_thres보다 높을 경우에만 append
            bbox = getXyxy(item[:4]) #cx, cy, w, h를 x1, y1, x2, y2 포맷으로 변환
            this_result = bbox # -> [x1, y1, x2, y2]
            this_result.append(this_conf) # -> [x1, y1, x2, y2, obj conf.]
            candidate_list[this_class_no]["bboxes(xyxy)"].append(this_result)

    for class_no in range(class_len):
        if len(candidate_list[class_no]["bboxes(xyxy)"]) > 0:
            #처리한 bbox 결과를 numpy ndarray 형식으로 변환
            candidate_list[class_no]["bboxes(xyxy)"] = np.array(candidate_list[class_no]["bboxes(xyxy)"])
    return candidate_list

4. 위에서 만든 함수를 이용해 conf thres를 기준으로 바운딩박스 후보군을 추려냅니다.

In [ ]:
conf_thres = 0.25
candidate_list = getCandidatesByClass(pred, conf_thres) #conf_thres를 기준으로 필터링
candidate_list # -> 클래스 개수만큼의 성분을 담은 리스트로 출력 [{0번 클래스 bbox정보}, {1번 클래스 bbox정보}]

In [ ]:
#클래스별 bbox 정보 출력
candidate_list[0] # -> { "class_no" : 0, "bboxes(xyxy)" : [...] }
print(len(candidate_list[0]["bboxes(xyxy)"])) # -> 0번 클래스(표지판)의 바운딩박스는 57개
print(len(candidate_list[1]["bboxes(xyxy)"])) # -> 1번 클래스(표지판)의 바운딩박스는 39개

5. 추려낸 후보군의 바운딩박스를 이미지로 그려 확인해 보겠습니다.

In [ ]:
class_names = ["sign", "light"]
bbox_colors = [(0, 255, 0), (0, 0, 255)] #표지판은 초록색으로, 신호등은 붉은색으로 표현

#이미지가 크므로 리사이즈한다.
dst = cv2.resize(src, (int((900*src.shape[1])/src.shape[0]), 900), interpolation=cv2.INTER_LINEAR)
for idx, candidate in enumerate(candidate_list): # -> candidate_list : [{0번 클래스 bbox정보}, {1번 클래스 bbox정보}]
    class_name = class_names[idx]
    color = bbox_colors[idx]
    bboxes = candidate["bboxes(xyxy)"]
    for bbox in bboxes:
        conf = bbox[-1]

        x1 = int(bbox[0] * dst.shape[1])
        y1 = int(bbox[1] * dst.shape[0])
        x2 = int(bbox[2] * dst.shape[1])
        y2 = int(bbox[3] * dst.shape[0])

        dst = cv2.rectangle(dst, (x1, y1), (x2, y2), color, 2)
        dst = cv2.putText(dst, "{}:{:.2f}".format(class_name, conf), (x1, y1-5), cv2.FONT_HERSHEY_DUPLEX, 0.6, color, 1)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

확률값이 0.25보다 낮은 바운딩박스는 모두 제거되어 이전보다 매우 깔끔해진 결과를 볼 수 있습니다. 하지만, 여전히 하나의 오브젝트에 너무 많은 바운딩박스가 겹쳐진 채로 출력되고 있습니다. 이제 Non Maximum Suppression을 통해 겹쳐진 바운딩박스들을 하나로 정리하는 작업이 필요합니다.

### Non Maximun Suppression을 통해 겹쳐진 바운딩박스 정리하기

1. 바운딩박스 2개의 IoU를 계산하는 함수를 먼저 만들어 놓겠습니다.

In [ ]:
#IoU : Intersection over Union
def IoU(box1, box2): # box1 : [x1, y1, x2, y2], box2 : [x1, y1, x2, y2]
    #2개의 박스가 서로 겹쳐져 있는지 확인하고, 겹쳐져 있지 않다면 0을 출력하게 하고 종료한다.
    if box1[2] < box2[0] or box1[0] > box2[2] or box1[3] < box2[1] or box1[1] > box2[3]:
        return 0

    #박스별 면적을 계산한다. 길이가 0~1 사이 실수값이므로 1을 더해 계산한다.
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    
    #2개 박스의 중첩되는 영역(교집합; Intersection)의 좌표를 구한다.
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    #중첩된 박스(Intersection)의 너비와 높이를 구한다.
    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    #중첩된 박스(Intersection)의 면적을 계산한다.
    inter = w * h

    #전체 박스 면적(Union) 중 중첩된 박스(Intersection)가 차지하는 비중을 구한다. -> IoU
    iou = inter / (box1_area + box2_area - inter)
    
    return iou

2. bbox 리스트를 입력으로 받아 IoU Threshold 값을 기준으로 Non Maximum Suppression을 진행하는 함수를 만들겠습니다.

In [ ]:
#겹치는 면적이 iou_thres를 넘으면 하나만 선택하도록 함
#bboxes : 바운딩박스 정보들을 담은 2차원 numpy 행렬 [[x1, y1, x2, y2, conf], [x1, y1, x2, y2, conf], [x1, y1, x2, y2, conf], ...]
#iou_thres : 바운딩박스 2개의 겹쳐진 부분의 면적비중이 iou_thres 값을 넘으면 둘 중 확률값이 높은 바운딩박스 하나만 선택하게 됨
def nms(bboxes, iou_thres = 0.45):
    if len(bboxes) == 0: #입력으로 들어온 bboxes의 개수가 0이라면 bboxes를 그대로 되돌려주고 종료 (에러방지)
        return bboxes

    #원본 바운딩박스 정보를 담고 있는 변수 B 생성 (=numpy ndarray)
    B = bboxes

    #2개의 바운딩박스 비교를 위해 임시로 사용하는 변수 T 생성
    T = None

    #검사가 완료된 바운딩박스만 담을 변수 D 생성
    D = None

    #B 중에서 conf score가 가장 높은 바운딩박스를 선택하고 T에 입력
    argmax_B = B[:, -1].argmax()
    T = B[argmax_B]

    #입력한 바운딩박스 정보를 B에서 삭제
    B = np.delete(B, argmax_B, axis=0)
    

    #T에 있는 바운딩박스와 B에 담겨 있는 모든 바운딩박스와의 IoU을 계산한 후 iou_thres보다 높은 면적을 가진 바운딩박스가 있다면 B에서 제거
    #이후 B에 남아있는 바운딩박스들 중 가장 높은 conf score를 가진 바운딩박스 하나를 뽑아 T에 다시 저장하고 반복
    while True:
        idx_to_delete = [] #B에서 삭제할 바운딩박스의 index 값을 담을 리스트 초기화
        for idx, B_item in enumerate(B): #B에 담긴 바운딩박스들을 반복문을 통해 하나씩 접근하면서 IoU 계산, 삭제여부 판단
            iou = IoU(T[:4], B_item[:4]) #T에 담긴 바운딩박스와 B의 idx번째 바운딩박스의 IoU 계산
            if iou > iou_thres: # IoU 가 주어진 임계치 iou_thres보다 크다면, idx 값을 idx_to_delete 리스트에 입력(T와 B의 idx번째 바운딩박스는 서로 겹쳐져 있다는 뜻)
                idx_to_delete.append(idx)
        B = np.delete(B, idx_to_delete, axis=0) #idx_to_delete에 입력된 idx번째 바운딩박스들을 B에서 모두 제거
        if D is None:
            D = T.copy() #T의 shape은 (5,)
            D = D.reshape(1, 5) #(5,)에 축을 추가해 D를 2차원으로 변환 (n행 5열이 될 수 있도록)
        else:
            D = np.vstack([D, T]) #D의 shape은 (n, 5), 즉 n행 5열의 shape이므로 vstack을 통해 T를 하나의 행으로하여 D에 append

        #B에 아직 바운딩박스가 남아 있다면, 남아있는 바운딩박스들 중 가장 높은 conf score를 가진 바운딩박스 하나를 뽑아 T에 다시 저장하고 반복
        if len(B) > 0:
            argmax_B = B[:, -1].argmax()
            T = B[argmax_B] #B에 남아있는 바운딩박스 정보들 중 가장 높은 conf score를 가진 바운딩박스 하나를 선택
            B = np.delete(B, argmax_B, axis=0) #T에 입력한 B의 바운딩박스 정보를 B에서 삭제
        
        #B에 바운딩박스가 남아 있지 않다면 반복문을 종료
        else:
            break
    
    #최종값 D 리턴
    return D

3. 위에서 conf_thres로 추려낸 바운딩박스에 NMS를 적용한 후 결과를 확인해 보겠습니다.

In [ ]:
bboxes_nms_0 = nms(candidate_list[0]["bboxes(xyxy)"]) # 57개로 추려졌던 0번 클래스(표지판)의 바운딩박스에 nms를 적용
bboxes_nms_1 = nms(candidate_list[1]["bboxes(xyxy)"]) # 39개로 추려졌던 1번 클래스(신호등)의 바운딩박스에 nms를 적용
print(len(bboxes_nms_0)) # -> 57개 바운딩박스가 5개로 정리
print(len(bboxes_nms_1)) # -> 39개 바운딩박스가 3개로 정리

In [ ]:
#실제 눈으로 확인
dst = cv2.resize(src, (int((900*src.shape[1])/src.shape[0]), 900), interpolation=cv2.INTER_LINEAR)
for idx, candidate in enumerate(candidate_list): # -> candidate_list : [{0번 클래스 bbox정보}, {1번 클래스 bbox정보}]
    class_name = class_names[idx]
    color = bbox_colors[idx]
    bboxes = nms(candidate["bboxes(xyxy)"]) #이 부분에서 nms를 적용
    for bbox in bboxes:
        conf = bbox[-1]

        x1 = int(bbox[0] * dst.shape[1])
        y1 = int(bbox[1] * dst.shape[0])
        x2 = int(bbox[2] * dst.shape[1])
        y2 = int(bbox[3] * dst.shape[0])

        dst = cv2.rectangle(dst, (x1, y1), (x2, y2), color, 2)
        dst = cv2.putText(dst, "{}:{:.2f}".format(class_name, conf), (x1, y1-5), cv2.FONT_HERSHEY_DUPLEX, 0.6, color, 1)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

다른 클래스의 바운딩박스가 서로 겹쳐져 있을 때는 어떻게 할 것인지, 작은 바운딩박스가 여러개 겹쳐져 있을 때 Merging을 어떻게 할 것인지 등 다양한 케이스가 존재할텐데 이러한 추가적인 후처리는 학습목적상 생략하였습니다. 추후 스스로 구현해 보기 바랍니다.

### 동영상에 적용해 결과 보기

In [ ]:
conf_thres = 0.25
class_names = ["sign", "light"]
bbox_colors = [(0, 255, 0), (0, 0, 255)]

# cap = cv2.VideoCapture(0) #카메라로 열 경우
cap = cv2.VideoCapture("test_images/test_video.mp4")

while True:
    retval, frame = cap.read()
    if retval is False:
        break

    src = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    src = cv2.resize(src, (640, 640))
    src = src/255.
    src = src.reshape((1, 640, 640, 3))
    pred = model(src)
    pred = pred[0][0].numpy()

    candidate_list = getCandidatesByClass(pred, conf_thres)

    dst = frame.copy()
    for idx, candidate in enumerate(candidate_list):
        class_name = class_names[idx]
        color = bbox_colors[idx]
        bboxes = nms(candidate_list[idx]["bboxes(xyxy)"])
        for bbox in bboxes:
            conf = bbox[-1]

            x1 = int(bbox[0] * dst.shape[1])
            y1 = int(bbox[1] * dst.shape[0])
            x2 = int(bbox[2] * dst.shape[1])
            y2 = int(bbox[3] * dst.shape[0])

            dst = cv2.rectangle(dst, (x1, y1), (x2, y2), color, 2)
            dst = cv2.putText(dst, "{}:{:.2f}".format(class_name, conf), (x1, y1-5), cv2.FONT_HERSHEY_DUPLEX, 0.6, color, 1)
    cv2.imshow("dst", dst)
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
cap.release()

### Lessons

1. 물체감지는 라벨링 작업에 시간이 많이 걸리므로 이번 실습은 일반에 공개된 데이터를 활용해 진행할 수 밖에 없었습니다. 현업에서 실제 활용할 목적으로 모델을 만들고자 한다면, 이미지 수집과 함께 라벨링 작업을 진행해 주어야 합니다.

2. 모델을 만드는 것 보다 데이터 전처리/후처리가 더 힘들고 공력이 많이 들어갑니다. 알고리즘에 대한 이해도 중요하지만, 전처리/후처리를 위한 코딩 기본기도 잘 닦아 놓기 바랍니다.